In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras as tfk

from sklearn.preprocessing import StandardScaler
from keras.callbacks import Callback

from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, LayerNormalization, RepeatVector

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
plt.style.use('seaborn')

from tensorflow.keras.preprocessing import timeseries_dataset_from_array

from utils import preprocess_time_series_v2

Using TensorFlow backend.


In [2]:
df = pd.read_csv('REE_2016_2020_gluonTS.csv')


In [3]:
df = pd.read_csv('REE_2016_2020_gluonTS.csv')

X_train, y_train, X_val, y_val, X_test, y_test, scaler = preprocess_time_series_v2(df, date_init='2016', date_final='2017', date_val='2018', date_test='2019', 
                            past_timestamps=96, forecasting_horizon=6, 
                            standarize=True, metafeatures=False, tendency=False)

                      Date  value
0      2016-01-02 23:00:00  24151
1      2016-01-03 00:00:00  22170
2      2016-01-03 01:00:00  20691
3      2016-01-03 02:00:00  19737
4      2016-01-03 03:00:00  19437
...                    ...    ...
34940  2019-12-28 19:00:00  29282
34941  2019-12-28 20:00:00  29266
34942  2019-12-28 21:00:00  28022
34943  2019-12-28 22:00:00  25964
34944  2019-12-28 23:00:00  24680

[34945 rows x 2 columns]
(17497, 1)
(8760, 1)
(8688, 1)
shape train/val (17497, 1) (8760, 1) (8688, 1)
Tamano de la serie temporal 2
Tamano de la serie temporal 2
/home/blasting/TFG_TimeSeries/notebooks/utils.py:133: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  val_data = df[date_val].to_numpy()
/home/blasting/TFG_TimeSeries/notebooks/utils.py:134: FutureWarning: Indexing a DataFrame with a datetimelike index using a 

In [10]:
input_train = tfk.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
 100, dropout=0.1, recurrent_dropout=0.1, 
 return_sequences=True, return_state=True)(input_train)
encoder_last_h = tfk.layers.BatchNormalization(momentum=0.6)(encoder_last_h)
encoder_last_c = tfk.layers.BatchNormalization(momentum=0.6)(encoder_last_c)

decoder_input = RepeatVector(y_train.shape[1])(encoder_last_h)
decoder_stack_h = LSTM(100, dropout=0.1, recurrent_dropout=0.1, return_state=False, return_sequences=True)(
    decoder_input, initial_state=[encoder_last_h, encoder_last_c])
print(decoder_stack_h)
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)
#print(attention)
context = dot([attention, encoder_stack_h], axes=[2,1])
context = BatchNormalization(momentum=0.6)(context)
#print(context)
decoder_combined_context = concatenate([context, decoder_stack_h])
#print(decoder_combined_context)

outputs = TimeDistributed(Dense(1))(decoder_combined_context)
model = tfk.Model(inputs=input_train, outputs=outputs)
print(model.summary)

model.compile(loss=tfk.losses.MeanSquaredError(), optimizer= tfk.optimizers.Adam(),
metrics=[tfk.metrics.MeanAbsoluteError()])
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 6, 100), dtype=tf.float32, name=None), name='lstm_11/transpose_1:0', description="created by layer 'lstm_11'")
<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7fbd35ed2c90>>
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 96, 1)]      0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, 96, 100), (N 40800       input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 100)          400         lstm_10[0][1]                    
_______________________

In [17]:
batch_size = 256
history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=2, 
                    validation_data=(X_val, y_val)) 


loss = history.history['loss']            #entrenamiento
val_loss = history.history['val_loss']    #validacion
#val_mse = history.history['val_mse']    #validacion
#mse = history.history['mse']    #validacion


Epoch 1/2
68/68 [==============================] - 36s 529ms/step - loss: 0.0241 - mean_absolute_error: 0.1119 - val_loss: 0.0215 - val_mean_absolute_error: 0.1026
Epoch 2/2
68/68 [==============================] - 37s 538ms/step - loss: 0.0242 - mean_absolute_error: 0.1118 - val_loss: 0.0228 - val_mean_absolute_error: 0.0974


In [18]:
def inverse_transform(yhat, y, scaler):
	return scaler.inverse_transform(y),  scaler.inverse_transform(yhat)

In [19]:
y_hat = model.predict(X_test)
y_hat_2 = model.predict(X_val)

y_test2, y_hat = inverse_transform(y_hat, y_test, scaler)
y_val2, y_hat_2 = inverse_transform(y_hat_2, y_val, scaler)

In [20]:
from sklearn.metrics import mean_absolute_error
def evaluate_forecasts(test, forecasts, n_seq):
	for i in range(n_seq):
		actual = [row[i] for row in test]
		predicted = [forecast[i] for forecast in forecasts]
		rmse = mean_absolute_error(actual, predicted)
		print('t+%d MSE: %f' % ((i+1), rmse))

In [21]:
print("test")
evaluate_forecasts(y_test2, y_hat, 6)

print("validacion")
evaluate_forecasts(y_val2, y_hat_2, 6)

test
t+1 MSE: 348.412469
t+2 MSE: 390.778849
t+3 MSE: 422.100241
t+4 MSE: 458.014382
t+5 MSE: 499.975009
t+6 MSE: 541.338184
validacion
t+1 MSE: 342.744563
t+2 MSE: 385.648673
t+3 MSE: 415.607388
t+4 MSE: 453.545182
t+5 MSE: 495.023425
t+6 MSE: 532.757518


In [22]:
(348.412469 + 390.778849 + 422.100241 + 458.014382 + 499.975009 + 541.338184)/6

443.43652233333324